In [8]:
!nvidia-smi

Tue Oct 15 13:45:35 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       On  |   00000000:00:1E.0 Off |                    0 |
| N/A   25C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:
import json 

instruct_dataset = f'../llava_instruct_150k.json'
with open(instruct_dataset, 'r') as f:
    instruct_data = json.load(f)

In [3]:
instruct_data[:10]

[{'id': '000000033471',
  'image': '000000033471.jpg',
  'conversations': [{'from': 'human',
    'value': '<image>\nWhat are the colors of the bus in the image?'},
   {'from': 'gpt', 'value': 'The bus in the image is white and red.'},
   {'from': 'human',
    'value': 'What feature can be seen on the back of the bus?'},
   {'from': 'gpt', 'value': 'The back of the bus features an advertisement.'},
   {'from': 'human',
    'value': 'Is the bus driving down the street or pulled off to the side?'},
   {'from': 'gpt',
    'value': 'The bus is driving down the street, which is crowded with people and other vehicles.'}]},
 {'id': '000000052846',
  'image': '000000052846.jpg',
  'conversations': [{'from': 'human',
    'value': 'Where is the cat positioned in the image?\n<image>'},
   {'from': 'gpt',
    'value': 'The cat is positioned on top of the back of the couch in the living room.'},
   {'from': 'human', 'value': 'What is the cat doing in the image?'},
   {'from': 'gpt',
    'value': 'Th

In [1]:
import json
from datasets import load_dataset
from typing import List, Dict

class ImageCaptionPreprocessor:
    def __init__(self, dataset_path: str = "data"):
        self.dataset = load_dataset("json", data_files=dataset_path, split="train")
        self.train_data = self.dataset['train']

    def extract_captions(self) -> List[Dict[str, str]]:
        captions = []
        for item in self.train_data:
            conversation = item['conversations']
            image_id = item['id']
            
            # Extract the caption (human's first message)
            caption = self._extract_caption_from_conversation(conversation)
            
            if caption:
                captions.append({
                    'image_id': image_id,
                    'caption': caption
                })
        
        return captions

    def _extract_caption_from_conversation(self, conversation: List[Dict[str, str]]) -> str:
        # The caption is typically the first human message
        for turn in conversation:
            if turn['from'] == 'human':
                # Remove any prefixes like "Human: " if present
                caption = turn['value'].lstrip("Human: ").strip()
                return caption
        return ""

    def save_captions_to_file(self, output_file: str = "data/preprocessed_captions.json"):
        captions = self.extract_captions()
        with open(output_file, 'w') as f:
            json.dump(captions, f, indent=2)
        print(f"Saved {len(captions)} captions to {output_file}")

    def get_sample_captions(self, n: int = 5) -> List[Dict[str, str]]:
        captions = self.extract_captions()
        return captions[:n]

# Usage example
if __name__ == "__main__":
    preprocessor = ImageCaptionPreprocessor()
    
    # Get a few sample captions
    samples = preprocessor.get_sample_captions(5)
    print("Sample captions:")
    for sample in samples:
        print(f"Image ID: {sample['image_id']}")
        print(f"Caption: {sample['caption']}\n")
    
    # Save all captions to a file
    preprocessor.save_captions_to_file()

FileNotFoundError: Unable to find '/home/ubuntu/multimodal-system/notebooks/data/llava_instruct_150k.json'